In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

template = """Question: {question}

Answer: Let's think step by step."""

prompt = ChatPromptTemplate.from_template(template)

model = OllamaLLM(model="qwen3:4b")

chain = prompt | model

response = chain.invoke({"question": "Hola, qué modelo eres?"}).split('</think>')[1].replace('\n','')


'\n\nHola, soy Qwen3, el último modelo de la serie Qwen desarrollado por Tongyi Lab. Tengo la capacidad de generar textos, escribir código y más. ¿En qué puedo ayudarte?'

In [16]:
import os
import weaviate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

def initialize_and_retrieve():
    client = weaviate.connect_to_local()
    
    docs = PyPDFLoader("./data/ejemplo_rag.pdf").load()
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=350, chunk_overlap=50
    )
    chunked_documents = text_splitter.split_documents(docs)

    collection_name = "DocumentosPDF"
    
    if client.collections.exists(collection_name):
        client.collections.delete(collection_name)
    
    collection = client.collections.create(
        name=collection_name,
        vectorizer_config=weaviate.classes.config.Configure.Vectorizer.text2vec_ollama(
            model="nomic-embed-text"
        ),
        generative_config=weaviate.classes.config.Configure.Generative.ollama(
            model="nomic-embed-text"
        )
    )
    
    with collection.batch.dynamic() as batch:
        for doc in chunked_documents:
            batch.add_object(
                properties={"text": doc.page_content}
            )

    response = collection.query.near_text(
        query="What is a RAG?",
        limit=2
    )
    
    for obj in response.objects:
        print(f"Propiedades: {obj.properties}")

# Llama a la función para ejecutar el código
initialize_and_retrieve()

c:\Users\Jose\Documents\Programacion\Chatbot_Langchain\.venv\Lib\site-packages\weaviate\warnings.py:196: DeprecationWarning: Dep024: You are using the `vectorizer_config` argument in `collection.config.create()`, which is deprecated.
            Use the `vector_config` argument instead.
            
  warnings.warn(


UnexpectedStatusCodeError: Collection may not have been created properly.! Unexpected status code: 422, with response body: {'error': [{'message': 'vectorizer: no module with name "text2vec-ollama" present'}]}.

In [1]:
from functions import check_ollama_connection
check_ollama_connection()

🤖 Modelos de Ollama disponibles:
   - all-minilm:33m
   - nomic-embed-text:latest
   - qwen3:4b


True